In [8]:
using StatsBase
using Combinatorics

include("jl/omega.jl")
include("jl/HSBM.jl")
include("jl/inference.jl")
include("jl/objectives.jl")


logLikelihood (generic function with 2 methods)

In [3]:
## Generate a graph
n = 200
Z = rand(1:3, n)
ϑ = dropdims(ones(1,n) + rand(1,n), dims = 1)

# defining group intensity function Ω
μ = mean(ϑ)

ω(p,α) = (10 .*μ*sum(p))^(-sum(p))*prod(p.^α)^(1/(sum(p)*α))
α0 = 1

kmax = 3

Ω = buildΩ(ω, α0, kmax)

H = sampleSBM(Z, ϑ, Ω; α=α0, kmax=kmax, kmin = 1)

hypergraph
  N: Array{Int64}((200,)) [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  191, 192, 193, 194, 195, 196, 197, 198, 199, 200]
  E: Dict{Int64,Dict}
  D: Array{Int64}((200,)) [7, 9, 3, 5, 7, 5, 2, 7, 13, 3  …  8, 11, 7, 9, 3, 9, 7, 10, 5, 2]


In [4]:
## Or, load one of the real world datasets
include("jl/read_data.jl")
kmax = 4
H,labels = read_hypergraph_data("congress-bills",kmax)
H,labels = read_hypergraph_data("contact-primary-school",kmax)
n = length(labels)

242

In [10]:
## Warm start using Clique-expansion Louvain
include("jl/warmstart.jl")
using SparseArrays, LinearAlgebra
c_warm = CliqueExpansionModularity(H)
Zsing = collect(1:n)
Ω = estimateΩEmpirically(H, c_warm)
mwarm = modularity(H,c_warm,Ω,α=α0)

-214929.2375688453280698179324833138363700801833289531447208500845435369397407044

In [17]:
## Run Louvain Algorithms
include("jl/fast_hypergraph_louvain.jl")
@time Zlou = HyperLouvain(H,kmax,Ω;α=α0)
@time Zlou0 = HyperLouvain_0(H,kmax,Ω;α=α0) # uses compute_moddiff
@time Znew, changed = SuperNodeStep(H,Zsing,kmax,Ω;α=α0)

mlou = modularity(H,Zlou,Ω,α=α0)
mlou0 = modularity(H,Zlou0,Ω,α=α0)
msu = modularity(H,Znew,Ω,α=α0)


pwd() = "/Users/nateveldt/GitHubRepos/hypergraph_modularities_code"
Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Louvain Iteration 4
Louvain Iteration 5
Louvain Iteration 6
  6.661642 seconds (44.91 M allocations: 3.973 GiB, 9.88% gc time)
Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Louvain Iteration 4
Louvain Iteration 5
Louvain Iteration 6
  7.414252 seconds (42.75 M allocations: 4.168 GiB, 9.41% gc time)
toc = 0.06824898719787598
Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Louvain Iteration 4
Louvain Iteration 5
Louvain Iteration 6
t1 = 6.689685344696045
  7.378747 seconds (42.61 M allocations: 4.216 GiB, 10.16% gc time)


-212723.0258701781403992419598633150354379188758818913392504001145036363062956752

In [ ]:
## Try earlier version
include("jl/hypergraph_louvain.jl")
@time Zlou = HyperLouvain(H,kmax,Ω;α=α0)
@time Znew, changed = SuperNodeStep(H,Zsing,kmax,Ω;α=α0)
msup = modularity(H,Znew,Ω,α=α0)
mlou = modularity(H,Zlou,Ω,α=α0)


Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Louvain Iteration 4
Louvain Iteration 5
Louvain Iteration 6
 13.049563 seconds (109.09 M allocations: 10.466 GiB, 12.82% gc time)

Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Louvain Iteration 4
Louvain Iteration 5